# YouTube Scraper - Testing Notebook

Test the YouTube scraper implementation:
1. **YouTubeScraper** - URL-based extraction (videos, channels, comments)
2. **YouTubeSearchScraper** - Parameter-based discovery with `extra_params`

---

## Setup - Use Local Development Version

In [1]:
import os
import sys
from pathlib import Path

# Add local src to path (use development version, not installed)
project_root = Path.cwd().parent
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print(f"Using source from: {src_path}")

# Load environment variables
from dotenv import load_dotenv
load_dotenv(project_root / ".env")

# Get API token
API_TOKEN = os.getenv("BRIGHTDATA_API_TOKEN")
if not API_TOKEN:
    raise ValueError("BRIGHTDATA_API_TOKEN not found in environment")

print(f"API Token: {API_TOKEN[:10]}...{API_TOKEN[-4:]}")
print("Setup complete!")

Using source from: /Users/ns/Desktop/projects/sdk-python/src
API Token: 7011787d-2...3336
Setup complete!


## Import YouTube Scrapers

In [2]:
from brightdata import BrightDataClient

# Verify we're using local version
import brightdata
print(f"brightdata module location: {brightdata.__file__}")

# Initialize client
client = BrightDataClient(token=API_TOKEN)

# Verify YouTube scraper is accessible
print(f"\nYouTubeScraper: {type(client.scrape.youtube).__name__}")
print(f"YouTubeSearchScraper: {type(client.search.youtube).__name__}")

# Check for scraper methods
print("\nScraper methods (URL-based):")
print([m for m in dir(client.scrape.youtube) if not m.startswith('_') and callable(getattr(client.scrape.youtube, m))])

print("\nSearch scraper methods (Discovery):")
print([m for m in dir(client.search.youtube) if not m.startswith('_') and callable(getattr(client.search.youtube, m))])

brightdata module location: /Users/ns/Desktop/projects/sdk-python/src/brightdata/__init__.py

YouTubeScraper: YouTubeScraper
YouTubeSearchScraper: YouTubeSearchScraper

Scraper methods (URL-based):
['channels', 'channels_fetch', 'channels_fetch_sync', 'channels_status', 'channels_status_sync', 'channels_sync', 'channels_trigger', 'channels_trigger_sync', 'comments', 'comments_fetch', 'comments_fetch_sync', 'comments_status', 'comments_status_sync', 'comments_sync', 'comments_trigger', 'comments_trigger_sync', 'normalize_result', 'scrape', 'scrape_async', 'videos', 'videos_fetch', 'videos_fetch_sync', 'videos_status', 'videos_status_sync', 'videos_sync', 'videos_trigger', 'videos_trigger_sync']

Search scraper methods (Discovery):
['channels_by_keyword', 'channels_by_keyword_sync', 'videos_by_channel', 'videos_by_channel_sync', 'videos_by_explore', 'videos_by_explore_sync', 'videos_by_hashtag', 'videos_by_hashtag_sync', 'videos_by_keyword', 'videos_by_keyword_sync', 'videos_by_search_fi

---
# Part 1: YouTubeScraper (URL-based Extraction)

Test URL-based extraction methods using `await` (required in Jupyter).

## 1.1 Videos - Extract video by URL

In [ ]:
# Test video extraction by URL
VIDEO_URL = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"

print(f"Scraping video: {VIDEO_URL}")
print("This may take 1-3 minutes...\n")

async with client.scrape.youtube.engine:
    result = await client.scrape.youtube.videos(url=VIDEO_URL, timeout=240)

print(f"Success: {result.success}")
print(f"Status: {result.status}")
print(f"Snapshot ID: {result.snapshot_id}")
print(f"Cost: ${result.cost:.4f}" if result.cost else "Cost: N/A")

if result.success and result.data:
    print("\n--- Video Data ---")
    data = result.data
    print(f"Available keys: {list(data.keys()) if isinstance(data, dict) else 'N/A'}")
    print(f"\nTitle: {data.get('title', 'N/A')}")
    print(f"Channel: {data.get('channel', 'N/A')}")
    print(f"Views: {data.get('views', 'N/A')}")
    print(f"Likes: {data.get('likes', 'N/A')}")
    print(f"Duration: {data.get('duration', 'N/A')}")
    print(f"Upload Date: {data.get('upload_date', data.get('date_posted', 'N/A'))}")
    description = str(data.get('description', 'N/A') or 'N/A')
    print(f"Description: {description[:100]}...")
else:
    print(f"\nError: {result.error}")

In [ ]:
# Export raw data to JSON file for inspection
import json
from pathlib import Path

output_file = Path.cwd() / "youtube_video_result.json"

export_data = {
    "success": result.success,
    "status": result.status,
    "snapshot_id": result.snapshot_id,
    "cost": result.cost,
    "row_count": result.row_count,
    "data": result.data,
    "error": result.error,
}

with open(output_file, "w") as f:
    json.dump(export_data, f, indent=2, default=str)

print(f"Exported to: {output_file}")
print(f"\nData type: {type(result.data)}")
print(f"Data preview: {str(result.data)[:500]}...")

## 1.2 Videos with Transcription

Use full language names like "English", "German", "Spanish", etc.

In [ ]:
# Test video extraction with transcription
VIDEO_URL = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"

print(f"Scraping video with transcription: {VIDEO_URL}")
print("Transcription language: English")
print("This may take 1-3 minutes...\n")

async with client.scrape.youtube.engine:
    result = await client.scrape.youtube.videos(
        url=VIDEO_URL,
        transcription_language="English",  # Use full language name, not code
        timeout=240
    )

print(f"Success: {result.success}")
print(f"Status: {result.status}")
print(f"Cost: ${result.cost:.4f}" if result.cost else "Cost: N/A")

if result.success and result.data:
    data = result.data
    print(f"\nTitle: {data.get('title', 'N/A')}")
    transcript = data.get('transcript', data.get('transcription', 'N/A'))
    if transcript and transcript != 'N/A':
        print(f"Transcript preview: {str(transcript)[:200]}...")
    else:
        print("No transcript available")
else:
    print(f"\nError: {result.error}")

## 1.3 Channels - Extract channel by URL

In [ ]:
# Test channel extraction by URL
CHANNEL_URL = "https://www.youtube.com/@MrBeast/about"

print(f"Scraping channel: {CHANNEL_URL}")
print("This may take 1-3 minutes...\n")

async with client.scrape.youtube.engine:
    result = await client.scrape.youtube.channels(url=CHANNEL_URL, timeout=240)

print(f"Success: {result.success}")
print(f"Status: {result.status}")
print(f"Snapshot ID: {result.snapshot_id}")
print(f"Cost: ${result.cost:.4f}" if result.cost else "Cost: N/A")

if result.success and result.data:
    print("\n--- Channel Data ---")
    data = result.data
    print(f"Available keys: {list(data.keys()) if isinstance(data, dict) else 'N/A'}")
    print(f"\nChannel Name: {data.get('channel_name', data.get('name', 'N/A'))}")
    print(f"Channel ID: {data.get('channel_id', 'N/A')}")
    print(f"Subscribers: {data.get('subscribers', data.get('subscriber_count', 'N/A'))}")
    print(f"Videos Count: {data.get('videos_count', data.get('video_count', 'N/A'))}")
    print(f"Total Views: {data.get('total_views', 'N/A')}")
    print(f"Joined Date: {data.get('joined_date', data.get('created_at', 'N/A'))}")
    description = str(data.get('description', data.get('about', 'N/A')) or 'N/A')
    print(f"Description: {description[:100]}...")
else:
    print(f"\nError: {result.error}")

In [ ]:
# Debug: See actual data structure
print("=== Raw Channel Data ===")
print(f"Type: {type(result.data)}")
print(f"Data: {result.data}")

## 1.4 Comments - Extract comments by video URL

In [ ]:
# Test comments extraction by video URL
VIDEO_URL = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"

print(f"Scraping comments from: {VIDEO_URL}")
print("Requesting 20 comments...")
print("This may take 1-3 minutes...\n")

async with client.scrape.youtube.engine:
    result = await client.scrape.youtube.comments(
        url=VIDEO_URL,
        num_of_comments=20,
        sort_by="Top comments",
        timeout=240
    )

print(f"Success: {result.success}")
print(f"Status: {result.status}")
print(f"Snapshot ID: {result.snapshot_id}")
print(f"Cost: ${result.cost:.4f}" if result.cost else "Cost: N/A")

if result.success and result.data:
    print("\n--- Comments Data ---")
    data = result.data
    if isinstance(data, list):
        print(f"Number of comments: {len(data)}")
        if len(data) > 0:
            print(f"Available keys: {list(data[0].keys())}")
        for i, comment in enumerate(data[:5]):
            print(f"\nComment {i+1}:")
            print(f"  User: {comment.get('username', 'N/A')}")
            text = str(comment.get('comment_text', 'N/A'))
            print(f"  Text: {text[:80]}...")
            print(f"  Likes: {comment.get('likes', 'N/A')}")
            print(f"  Replies: {comment.get('replies', 'N/A')}")
            print(f"  Date: {comment.get('date', 'N/A')}")
    elif isinstance(data, dict):
        print(f"Available keys: {list(data.keys())}")
        print(f"Data: {data}")
    else:
        print(f"Data type: {type(data)}")
else:
    print("\nNo data returned. Debug info:")
    print(f"  result.data: {result.data}")
    print(f"  result.row_count: {result.row_count}")
    print(f"  result.error: {result.error}")

---
# Part 2: YouTubeSearchScraper (Discovery with extra_params)

Test parameter-based discovery methods that use `extra_params` for:
- `type=discover_new`
- `discover_by=explore|hashtag|keyword|search_filters|channel|channel_search`

## 2.1 Videos Discovery - by Keyword

In [3]:
# Test videos discovery by keyword
# Uses: extra_params={"type": "discover_new", "discover_by": "keyword"}
KEYWORD = "python tutorial"

print(f"Discovering videos for keyword: {KEYWORD}")
print("Using extra_params: type=discover_new, discover_by=keyword")
print("Requesting 10 videos...")
print("This may take 1-3 minutes...\n")

async with client.search.youtube.engine:
    result = await client.search.youtube.videos_by_keyword(
        keyword=KEYWORD,
        num_of_posts=10,  # Note: parameter is num_of_posts, not num_of_videos
        timeout=240
    )

print(f"Success: {result.success}")
print(f"Status: {result.status}")
print(f"Snapshot ID: {result.snapshot_id}")
print(f"Cost: ${result.cost:.4f}" if result.cost else "Cost: N/A")

if result.success and result.data:
    print("\n--- Discovered Videos ---")
    data = result.data
    if isinstance(data, list):
        print(f"Number of videos discovered: {len(data)}")
        if len(data) > 0:
            print(f"Available keys: {list(data[0].keys()) if isinstance(data[0], dict) else 'N/A'}")
        for i, video in enumerate(data[:5]):
            print(f"\nVideo {i+1}:")
            print(f"  URL: {video.get('url', 'N/A')}")
            print(f"  Title: {video.get('title', 'N/A')}")
            print(f"  Channel: {video.get('youtuber', video.get('channel', 'N/A'))}")
            print(f"  Views: {video.get('views', 'N/A')}")
            print(f"  Duration: {video.get('video_length', video.get('duration', 'N/A'))}")
    elif isinstance(data, dict):
        print(f"Available keys: {list(data.keys())}")
    else:
        print(f"Data type: {type(data)}")
else:
    print(f"\nError: {result.error}")

Discovering videos for keyword: python tutorial
Using extra_params: type=discover_new, discover_by=keyword
Requesting 10 videos...
This may take 1-3 minutes...

Success: True
Status: ready
Snapshot ID: sd_mkrgw1tq5250tdejx
Cost: $0.0200

--- Discovered Videos ---
Number of videos discovered: 10
Available keys: ['url', 'title', 'youtuber', 'youtuber_md5', 'video_url', 'video_length', 'likes', 'views', 'date_posted', 'description', 'num_comments', 'subscribers', 'video_id', 'channel_url', 'preview_image', 'discovery_input', 'shortcode', 'verified', 'handle_name', 'avatar_img_channel', 'is_sponsored', 'related_videos', 'license', 'viewport_frames', 'current_optimal_res', 'codecs', 'color', 'quality', 'quality_label', 'post_type', 'youtuber_id', 'transcript', 'formatted_transcript', 'hashtags', 'tags', 'next_recommended_videos', 'recommended_videos', 'transcript_language', 'chapters', 'transcription_language', 'is_age_restricted', 'channel_url_decoded', 'timestamp', 'input']

Video 1:
  UR

## 2.2 Videos Discovery - by Hashtag

In [4]:
# Test videos discovery by hashtag
# Uses: extra_params={"type": "discover_new", "discover_by": "hashtag"}
HASHTAG = "coding"  # without # prefix

print(f"Discovering videos for hashtag: {HASHTAG}")
print("Using extra_params: type=discover_new, discover_by=hashtag")
print("Requesting 10 videos...")
print("This may take 1-3 minutes...\n")

async with client.search.youtube.engine:
    result = await client.search.youtube.videos_by_hashtag(
        hashtag=HASHTAG,
        num_of_posts=10,
        timeout=240
    )

print(f"Success: {result.success}")
print(f"Status: {result.status}")
print(f"Snapshot ID: {result.snapshot_id}")
print(f"Cost: ${result.cost:.4f}" if result.cost else "Cost: N/A")

if result.success and result.data:
    print("\n--- Discovered Videos ---")
    data = result.data
    if isinstance(data, list):
        print(f"Number of videos discovered: {len(data)}")
        for i, video in enumerate(data[:3]):
            print(f"\nVideo {i+1}:")
            print(f"  URL: {video.get('url', 'N/A')}")
            print(f"  Title: {video.get('title', 'N/A')}")
            print(f"  Channel: {video.get('youtuber', video.get('channel', 'N/A'))}")
    else:
        print(f"Data type: {type(data)}")
else:
    print(f"\nError: {result.error}")

Discovering videos for hashtag: coding
Using extra_params: type=discover_new, discover_by=hashtag
Requesting 10 videos...
This may take 1-3 minutes...

Success: True
Status: ready
Snapshot ID: sd_mkrh0klv1n58cbnc5h
Cost: $0.0200

--- Discovered Videos ---
Number of videos discovered: 10

Video 1:
  URL: https://www.youtube.com/shorts/yymyLXPbP1Y
  Title: How to become a Senior developer.. ♥️ 😂 #programming #javascript #python #coding #developer #coder .
  Channel: @divineclassesaninstitution5273

Video 2:
  URL: https://www.youtube.com/shorts/Y3Lmf5QoaKA
  Title: Created Reptile | HTML | CSS | Javascript #youtubeshorts #trending #coding #animation #ai #learning
  Channel: @shivin_tutorial

Video 3:
  URL: https://www.youtube.com/shorts/W-8j4MrsX2s
  Title: Amazing Flower Design using Python turtle 🐢 #python #coding #funny #viral #trending #design
  Channel: @dev.19.community


## 2.3 Videos Discovery - by Channel

In [5]:
# Test videos discovery from channel
# Uses: extra_params={"type": "discover_new", "discover_by": "url"}
CHANNEL_URL = "https://www.youtube.com/@MrBeast/videos"

print(f"Discovering videos from channel: {CHANNEL_URL}")
print("Using extra_params: type=discover_new, discover_by=url")
print("Requesting 10 videos...")
print("This may take 1-3 minutes...\n")

async with client.search.youtube.engine:
    result = await client.search.youtube.videos_by_channel(
        url=CHANNEL_URL,
        num_of_posts=10,
        timeout=240
    )

print(f"Success: {result.success}")
print(f"Status: {result.status}")
print(f"Snapshot ID: {result.snapshot_id}")
print(f"Cost: ${result.cost:.4f}" if result.cost else "Cost: N/A")

if result.success and result.data:
    print("\n--- Discovered Videos from Channel ---")
    data = result.data
    if isinstance(data, list):
        print(f"Number of videos discovered: {len(data)}")
        if len(data) > 0:
            print(f"Available keys: {list(data[0].keys()) if isinstance(data[0], dict) else 'N/A'}")
        for i, video in enumerate(data[:5]):
            print(f"\nVideo {i+1}:")
            print(f"  URL: {video.get('url', 'N/A')}")
            print(f"  Title: {video.get('title', 'N/A')}")
            print(f"  Views: {video.get('views', 'N/A')}")
            print(f"  Duration: {video.get('video_length', video.get('duration', 'N/A'))}")
    else:
        print(f"Data type: {type(data)}")
else:
    print(f"\nError: {result.error}")

Discovering videos from channel: https://www.youtube.com/@MrBeast/videos
Using extra_params: type=discover_new, discover_by=url
Requesting 10 videos...
This may take 1-3 minutes...

Success: True
Status: ready
Snapshot ID: sd_mkrh5lzi2m2w8vvw3d
Cost: $0.0200

--- Discovered Videos from Channel ---
Number of videos discovered: 10
Available keys: ['url', 'title', 'youtuber', 'youtuber_md5', 'video_url', 'video_length', 'likes', 'views', 'date_posted', 'description', 'num_comments', 'subscribers', 'video_id', 'channel_url', 'preview_image', 'discovery_input', 'shortcode', 'verified', 'handle_name', 'avatar_img_channel', 'is_sponsored', 'related_videos', 'license', 'viewport_frames', 'current_optimal_res', 'codecs', 'color', 'quality', 'quality_label', 'post_type', 'youtuber_id', 'transcript', 'formatted_transcript', 'hashtags', 'tags', 'next_recommended_videos', 'recommended_videos', 'transcript_language', 'chapters', 'transcription_language', 'is_age_restricted', 'channel_url_decoded', '

## 2.4 Channels Discovery - by Keyword

In [ ]:
# Test channels discovery by keyword
# Uses: extra_params={"type": "discover_new", "discover_by": "keyword"}
# Note: This endpoint doesn't have a num_of_channels limit parameter
KEYWORD = "tech review"

print(f"Discovering channels for keyword: {KEYWORD}")
print("Using extra_params: type=discover_new, discover_by=keyword")
print("This may take 1-3 minutes...\n")

async with client.search.youtube.engine:
    result = await client.search.youtube.channels_by_keyword(
        keyword=KEYWORD,
        timeout=240
    )

print(f"Success: {result.success}")
print(f"Status: {result.status}")
print(f"Snapshot ID: {result.snapshot_id}")
print(f"Cost: ${result.cost:.4f}" if result.cost else "Cost: N/A")

if result.success and result.data:
    print("\n--- Discovered Channels ---")
    data = result.data
    if isinstance(data, list):
        # Filter out error records (those without 'url' or 'name')
        valid_channels = [c for c in data if c.get('url') or c.get('name')]
        error_count = len(data) - len(valid_channels)
        
        print(f"Total records: {len(data)}")
        print(f"Valid channels: {len(valid_channels)}")
        print(f"Error records: {error_count}")
        
        if len(valid_channels) > 0:
            print(f"Available keys: {list(valid_channels[0].keys())}")
        for i, channel in enumerate(valid_channels[:5]):
            print(f"\nChannel {i+1}:")
            print(f"  URL: {channel.get('url', 'N/A')}")
            print(f"  Name: {channel.get('name', channel.get('channel_name', 'N/A'))}")
            print(f"  Subscribers: {channel.get('subscribers', 'N/A')}")
            print(f"  Videos: {channel.get('videos_count', 'N/A')}")
    else:
        print(f"Data type: {type(data)}")
else:
    print(f"\nError: {result.error}")

## 2.5 Videos Discovery - by Explore

In [8]:
# Test videos discovery from explore page
# Uses: extra_params={"type": "discover_new", "discover_by": "explore"}
# Note: This endpoint may take longer - using 660s (11 min) timeout
EXPLORE_URL = "https://www.youtube.com/gaming/games"

print(f"Discovering videos from YouTube explore page: {EXPLORE_URL}")
print("Using extra_params: type=discover_new, discover_by=explore")
print("This may take up to 11 minutes...\n")

async with client.search.youtube.engine:
    result = await client.search.youtube.videos_by_explore(
        url=EXPLORE_URL,
        timeout=660  # 11 minutes
    )

print(f"Success: {result.success}")
print(f"Status: {result.status}")
print(f"Snapshot ID: {result.snapshot_id}")
print(f"Cost: ${result.cost:.4f}" if result.cost else "Cost: N/A")

if result.success and result.data:
    print("\n--- Discovered Videos from Explore ---")
    data = result.data
    if isinstance(data, list):
        print(f"Number of videos discovered: {len(data)}")
        for i, video in enumerate(data[:3]):
            print(f"\nVideo {i+1}:")
            print(f"  URL: {video.get('url', 'N/A')}")
            print(f"  Title: {video.get('title', 'N/A')}")
            print(f"  Channel: {video.get('youtuber', video.get('channel', 'N/A'))}")
    else:
        print(f"Data type: {type(data)}")
else:
    print(f"\nError: {result.error}")

Discovering videos from YouTube explore page: https://www.youtube.com/gaming/games
Using extra_params: type=discover_new, discover_by=explore
This may take up to 11 minutes...

Success: False
Status: error
Snapshot ID: sd_mkri46dn27xrmpwu1d
Cost: N/A

Error: Failed to fetch results: Failed to fetch results (HTTP 202): {"status":"building","message":"Dataset is not ready yet, try again in 30s"}


## 2.6 Videos Discovery - by Search Filters

In [9]:
# Test videos discovery with search filters
# Uses: extra_params={"type": "discover_new", "discover_by": "search_filters"}
KEYWORD = "python programming"

print(f"Discovering videos with search filters for: {KEYWORD}")
print("Using extra_params: type=discover_new, discover_by=search_filters")
print("This may take 1-3 minutes...\n")

async with client.search.youtube.engine:
    result = await client.search.youtube.videos_by_search_filters(
        keyword_search=KEYWORD,
        upload_date="This month",
        video_type="Video",
        sort_by="View count",
        timeout=240
    )

print(f"Success: {result.success}")
print(f"Status: {result.status}")
print(f"Snapshot ID: {result.snapshot_id}")
print(f"Cost: ${result.cost:.4f}" if result.cost else "Cost: N/A")

if result.success and result.data:
    print("\n--- Discovered Videos from Search ---")
    data = result.data
    if isinstance(data, list):
        print(f"Number of videos discovered: {len(data)}")
        for i, video in enumerate(data[:3]):
            print(f"\nVideo {i+1}:")
            print(f"  URL: {video.get('url', 'N/A')}")
            print(f"  Title: {video.get('title', 'N/A')}")
            print(f"  Channel: {video.get('youtuber', video.get('channel', 'N/A'))}")
    else:
        print(f"Data type: {type(data)}")
else:
    print(f"\nError: {result.error}")

Discovering videos with search filters for: python programming
Using extra_params: type=discover_new, discover_by=search_filters
This may take 1-3 minutes...

Success: False
Status: error
Snapshot ID: sd_mkriunzgcw2yqszb7
Cost: N/A

Error: Failed to fetch results: Failed to fetch results (HTTP 202): {"status":"building","message":"Dataset is not ready yet, try again in 30s"}


---
# Part 3: Verify Timing Metadata

In [ ]:
# Check timing metadata from last result
print("=== Timing Metadata ===")
print(f"trigger_sent_at: {result.trigger_sent_at}")
print(f"snapshot_id_received_at: {result.snapshot_id_received_at}")
print(f"snapshot_polled_at: {result.snapshot_polled_at}")
print(f"data_fetched_at: {result.data_fetched_at}")
print(f"\nrow_count: {result.row_count}")
print(f"cost: {result.cost}")

---
# Summary

## YouTubeScraper (URL-based)
- `videos(url)` - Extract video data by URL (with optional transcription using full language names like "English")
- `channels(url)` - Extract channel data by URL
- `comments(url, num_of_comments)` - Extract comments from video URL

## YouTubeSearchScraper (Discovery with extra_params)
- `videos_by_explore(url)` - Discover from explore page (`discover_by=explore`)
- `videos_by_hashtag(hashtag, num_of_posts)` - Discover by hashtag (`discover_by=hashtag`)
- `videos_by_keyword(keyword, num_of_posts)` - Discover by keyword (`discover_by=keyword`)
- `videos_by_search_filters(keyword_search, ...)` - Discover with search filters (`discover_by=search_filters`)
- `videos_by_channel(url, num_of_posts)` - Discover from channel (`discover_by=url`)
- `channels_by_keyword(keyword)` - Discover channels (`discover_by=keyword`)

## Common Response Fields
- Video: `url`, `title`, `youtuber`, `views`, `likes`, `video_length`, `date_posted`, `description`
- Channel: `url`, `name`, `subscribers`, `videos_count`, `Description`
- Comments: `username`, `comment_text`, `likes`, `replies`, `date`